In [1]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import time
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pymysql
from io import StringIO

In [2]:
def start_driver():
    # Service 객체를 명시적으로 설정
    service = Service(ChromeDriverManager().install())

    # Chrome WebDriver 실행
    driver = webdriver.Chrome(service=service)

    # 웹페이지 접속
    driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx')

    # 드롭다운 요소가 로드될 때까지 기다리기
    wait = WebDriverWait(driver, 10)  # 최대 10초까지 대기
    return wait, driver

In [ ]:
# KBO 홈페이지 연도
kbo_years = [str(i) for i in range(1982, 2025)]
# KBO 팀들 딕셔너리
kbo_teams = {
    'OB': 'OB',
    '삼성': 'SS',
    'MBC': 'LG',
    '해태': 'HT',
    '롯데': 'LT',
    '삼미': 'HD',
    '청보': 'HD',
    '빙그레': 'HH',
    '태평양': 'HD',
    'LG': 'LG',
    '쌍방울': 'SB',
    '한화': 'HH',
    '현대': 'HD',
    '두산': 'OB',
    'SK': 'SK',
    'KIA': 'HT',
    '우리': 'WO',
    '넥센': 'WO',
    'NC': 'NC',
    'KT': 'KT',
    '키움': 'WO',
    'SSG': 'SK'}

In [ ]:
wait, driver = start_driver()
for kbo_year in kbo_years:
    # 연도 드롭다운 선택
    Select(driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlSeason_ddlSeason')).select_by_visible_text(kbo_year)
    time.sleep(.5)

    # 각 연도에서 팀들 추출해서 합치기
    dropdown_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')))
    kbo_teams.update({option.text:option.get_attribute("value") for option in Select(dropdown_element).options[1:]})
driver.quit()

In [17]:
wait, driver = start_driver()
data_df = []
# 페이지에서 테이블 가져오기
table_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.record_result > table')))

# 테이블의 HTML을 문자열로 변환 후, pandas로 읽기
table_html = table_element.get_attribute("outerHTML")
data_df.append(pd.read_html(table_html)[0])  # 첫 번째 테이블 가져오기
# pd.read_html(driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.record_result > table').get_attribute("outerHTML"))[0]

# 2페이지 찾기
second_page = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo2')
if second_page:
    second_page.click()
    
    time.sleep(.5)
    
    # 페이지에서 테이블 가져오기
    table_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.record_result > table')))

    # 테이블의 HTML을 문자열로 변환 후, pandas로 읽기
    table_html = table_element.get_attribute("outerHTML")
    data_df.append(pd.read_html(table_html)[0])  # 첫 번째 테이블 가져오기


/tmp/ipykernel_116620/2095380461.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_df.append(pd.read_html(table_html)[0])  # 첫 번째 테이블 가져오기
/tmp/ipykernel_116620/2095380461.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_df.append(pd.read_html(table_html)[0])  # 첫 번째 테이블 가져오기


In [29]:
for data in pd.concat(data_df, ignore_index=True).iterrows():
    pass
data[1].iloc[1:]

선수명      전준우
팀명        롯데
AVG    0.188
G          9
PA        36
AB        32
H          6
2B         2
3B         0
HR         0
RBI        2
SB         0
CS         0
BB         3
HBP        0
SO         3
GDP        0
E          0
Name: 59, dtype: object

In [9]:
wait, driver = start_driver()
driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx')

for kbo_year in kbo_years:
    # 연도 드롭다운 선택
    Select(driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlSeason_ddlSeason')).select_by_visible_text(kbo_year)
    time.sleep(.5)

    # 팀 드롭다운 선택
    dropdown_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')))
    kbo_teams = [option.get_attribute("value") for option in Select(dropdown_element).options[1:]]

    for kbo_team in kbo_teams:
        # 팀 선택하기
        Select(dropdown_element).select_by_value(kbo_team)

        # 페이지에서 테이블 가져오기
        table_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.record_result > table')))

        # 테이블의 HTML을 문자열로 변환 후, pandas로 읽기
        table_html = table_element.get_attribute("outerHTML")        
        data_df.append(pd.read_html(table_html)[0])  # 첫 번째 테이블 가져오기

        # 2페이지 찾기
        second_page = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo2')
        if second_page:
            second_page.click()
            
            time.sleep(.5)
            
            # 페이지에서 테이블 가져오기
            table_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_udpContent > div.record_result > table')))

            # 테이블의 HTML을 문자열로 변환 후, pandas로 읽기
            table_html = table_element.get_attribute("outerHTML")
            data_df.append(pd.read_html(table_html)[0])  # 첫 번째 테이블 가져오기


TypeError: 'NoneType' object is not subscriptable

In [5]:
import pymysql
# 데이터베이스 연결하기
conn = pymysql.connect(host='127.0.0.1', user='apic', password='1234', db='KBODATA', charset='utf8')
# 통로 만들기
cur = conn.cursor()
cur.execute('INSERT INTO ')

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '' at line 1")

In [20]:
wait, driver = start_driver()
driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx')
# KBO 홈페이지 연도
kbo_years = [str(i) for i in range(2020, 2022)]

# 데이터베이스 연결하기
conn = pymysql.connect(host='127.0.0.1', user='apic', password='1234', db='KBODATA', charset='utf8')

# 통로 만들기
cur = conn.cursor()

for kbo_year in kbo_years:
    # 연도 드롭다운 선택
    # Select(driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlSeason_ddlSeason')).select_by_visible_text(kbo_year)
    dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlSeason_ddlSeason'))
    )
    Select(dropdown).select_by_visible_text(kbo_year)
    time.sleep(.5)

    # 팀 드롭다운 선택
    dropdown_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')))
    kbo_teams = [option.get_attribute("value") for option in Select(dropdown_element).options[1:]]

    for kbo_team in kbo_teams:
        # print(kbo_year, kbo_team)
        time.sleep(.5)
        # 팀 선택하기
        dropdown_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ddlTeam_ddlTeam')))
        Select(dropdown_element).select_by_value(kbo_team)

        time.sleep(.5)
        # 페이지에서 테이블 가져오기
        table_element = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))

        # 테이블의 HTML을 문자열로 변환 후, pandas로 읽기
        table_html = table_element.get_attribute("outerHTML")
        df = pd.read_html(StringIO(table_html))[0]
        df['year'] = kbo_year
        
        # print(df)
        
        # mysql에 있는 컬럼명으로 바꾸기
        df.rename(columns={"선수명": "player_name"}, inplace=True)
        df.rename(columns={"팀명": "team_name"}, inplace=True)
        
        # - 문자 0으로 바꾸기
        df.replace('-', '0', inplace=True)
        
        
        # print(df)
        # print(df.columns[1:])
        for _, data in df.iterrows():
            # print(data.iloc[1:])
            columns = ", ".join(df.columns[1:])  # 컬럼 리스트를 문자열로 변환
            
            values_placeholder = ", ".join(["%s"] * len(data.iloc[1:]))  # 플레이스홀더 생성
            
            sql = f"INSERT INTO KBO_TABLE ({columns}) VALUES ({values_placeholder})"
            cur.execute(sql, tuple(data.iloc[1:]))  # 데이터를 튜플로 변환하여 실행

        try:
            # 2페이지 찾기
            second_page = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo2')
            if not second_page:
                continue
            second_page.click()
            time.sleep(.5)
            
            time.sleep(.5)
            # 페이지에서 테이블 가져오기
            table_element = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))

            # 테이블의 HTML을 문자열로 변환 후, pandas로 읽기
            table_html = table_element.get_attribute("outerHTML")
            df = pd.read_html(StringIO(table_html))[0]
            df['year'] = kbo_year
            df.rename(columns={"선수명": "player_name"}, inplace=True)
            df.rename(columns={"팀명": "team_name"}, inplace=True)
            
            df.replace('-', '0', inplace=True)
            # print(df)
            # print(df.columns[1:])
            for _, data in df.iterrows():
                columns = ", ".join(df.columns[1:])  # 컬럼 리스트를 문자열로 변환
                values_placeholder = ", ".join(["%s"] * len(data.iloc[1:]))  # 플레이스홀더 생성
                
                sql = f"INSERT INTO KBO_TABLE ({columns}) VALUES ({values_placeholder})"
                cur.execute(sql, tuple(data.iloc[1:]))  # 데이터를 튜플로 변환하여 실행
                
            # 첫 번째 페이지로 돌아가기
            first_page = driver.find_element(By.CSS_SELECTOR, '#cphContents_cphContents_cphContents_ucPager_btnNo1').click()
            time.sleep(.5)
        except NoSuchElementException:
            pass
    conn.commit()
    
driver.quit()

In [19]:
conn.close()